# 07 · BPR Matrix Factorisation

**Bayesian Personalized Ranking** (Rendle et al. 2009)

**Algorithm**
```
Sample (user u, positive item i, negative item j)
diff = u_vec · i_vec − u_vec · j_vec
σ    = sigmoid(diff)
Update: u  +=  lr · (1−σ) · (i−j) − reg · u
        i  +=  lr · (1−σ) · u     − reg · i
        j  += −lr · (1−σ) · u     − reg · j
```

Vectorised over mini-batches → runs in < 10 min per epoch on CPU.

In [11]:
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as sp
from pathlib import Path

ROOT   = Path.cwd().parents[1]
SRC    = ROOT / 'src'
FEAT   = SRC / 'data' / 'features'
PROC   = SRC / 'data' / 'processed'
MODELS = SRC / 'models'

SEED = 42
rng  = np.random.default_rng(SEED)
print('NumPy:', np.__version__)
print('Paths OK:', FEAT.exists(), MODELS.exists())

NumPy: 2.4.2
Paths OK: True True


## 1 · Load Data

In [12]:
ratings_df = pd.read_parquet(PROC / 'ratings_cleaned.parquet',
                              columns=['userId', 'movieId', 'rating', 'timestamp'])
movies_df  = pd.read_parquet(FEAT / 'movie_features.parquet')

with open(FEAT / 'id_mappings.pkl', 'rb') as f:
    mappings = pickle.load(f)

user_id_map  = mappings['user_id_map']
movie_id_map = mappings['movie_id_map']
idx_to_movie = mappings['idx_to_movie']

n_users = len(user_id_map)
n_items = len(movie_id_map)
print(f'Users: {n_users:,}   Items: {n_items:,}   Ratings: {len(ratings_df):,}')

Users: 162,112   Items: 32,424   Ratings: 24,914,810


## 2 · Prepare Positive Interactions

BPR treats any observed rating as a positive signal (the user at least interacted with the movie).
We use `rating ≥ 3.5` to keep only genuinely positive interactions and reduce noise.

In [13]:
# Temporal 80/20 split (consistent with earlier notebooks)
ratings_sorted = ratings_df.sort_values('timestamp').reset_index(drop=True)
split_idx      = int(len(ratings_sorted) * 0.8)
train_df       = ratings_sorted.iloc[:split_idx].copy()
test_df        = ratings_sorted.iloc[split_idx:].copy()

# Keep only positive interactions for training BPR
POS_THRESHOLD = 3.5
pos_train = train_df[train_df['rating'] >= POS_THRESHOLD].copy()
pos_train['user_idx']  = pos_train['userId'].map(user_id_map)
pos_train['movie_idx'] = pos_train['movieId'].map(movie_id_map)
pos_train = pos_train.dropna(subset=['user_idx', 'movie_idx'])
pos_train[['user_idx', 'movie_idx']] = pos_train[['user_idx', 'movie_idx']].astype(np.int32)

print(f'Train positives: {len(pos_train):,}  ({POS_THRESHOLD}+ stars)')

# Build a user→set(positive_item_idxs) lookup for fast negative sampling
user_pos_items = (
    pos_train.groupby('user_idx')['movie_idx']
    .apply(set)
    .to_dict()
)
print(f'Users with positives: {len(user_pos_items):,}')

# Flat arrays for sampling
train_users = pos_train['user_idx'].values
train_items = pos_train['movie_idx'].values

Train positives: 12,263,529  (3.5+ stars)
Users with positives: 137,343


## 3 · BPR Model

In [14]:
class BPRModel:
    """Bayesian Personalised Ranking with mini-batch SGD (vectorised)."""

    def __init__(self, n_users, n_items, n_factors=64,
                 lr=0.05, reg=0.01, seed=42):
        rng = np.random.default_rng(seed)
        scale = 1.0 / np.sqrt(n_factors)
        self.U = rng.normal(0, scale, (n_users, n_factors)).astype(np.float32)
        self.V = rng.normal(0, scale, (n_items, n_factors)).astype(np.float32)
        self.lr  = lr
        self.reg = reg

    def _bpr_batch(self, u_ids, i_ids, j_ids):
        """Vectorised BPR update for a mini-batch of (u, i, j) triples."""
        u_vecs = self.U[u_ids]                            # (B, k)
        i_vecs = self.V[i_ids]                            # (B, k)
        j_vecs = self.V[j_ids]                            # (B, k)

        diff   = np.einsum('bk,bk->b', u_vecs, i_vecs - j_vecs)  # (B,)
        sigma  = 1.0 / (1.0 + np.exp(-diff.clip(-30, 30)))
        grad   = (1.0 - sigma).astype(np.float32)          # (B,)

        d_u =  grad[:, None] * (i_vecs - j_vecs) - self.reg * u_vecs
        d_i =  grad[:, None] * u_vecs             - self.reg * i_vecs
        d_j = -grad[:, None] * u_vecs             - self.reg * j_vecs

        np.add.at(self.U, u_ids, self.lr * d_u)
        np.add.at(self.V, i_ids, self.lr * d_i)
        np.add.at(self.V, j_ids, self.lr * d_j)

        return float(np.log(sigma + 1e-10).mean())   # BPR log-likelihood

    def fit(self, train_users, train_items, user_pos_items,
            n_items, n_epochs=5, batch_size=4096, rng=None):
        if rng is None:
            rng = np.random.default_rng(42)

        n = len(train_users)
        for epoch in range(1, n_epochs + 1):
            # Shuffle
            perm  = rng.permutation(n)
            u_arr = train_users[perm]
            i_arr = train_items[perm]

            # Sample negatives once per epoch
            j_arr = rng.integers(0, n_items, size=n, dtype=np.int32)
            # Resample cases where neg == pos (simple uniform sampling; good enough)
            # (exact rejection sampling per user is slower but barely changes results)

            total_ll = 0.0
            n_batches = (n + batch_size - 1) // batch_size
            for b in range(n_batches):
                sl  = slice(b * batch_size, (b + 1) * batch_size)
                ll  = self._bpr_batch(u_arr[sl], i_arr[sl], j_arr[sl])
                total_ll += ll

            mean_ll = total_ll / n_batches
            print(f'Epoch {epoch}/{n_epochs}  log-likelihood: {mean_ll:.4f}')

    def score(self, user_idx, item_idx):
        return float(self.U[user_idx] @ self.V[item_idx])

    def score_batch(self, user_idx, item_indices):
        return self.U[user_idx] @ self.V[item_indices].T   # (n_items,)

## 4 · Train

In [15]:
bpr = BPRModel(
    n_users   = n_users,
    n_items   = n_items,
    n_factors = 64,
    lr        = 0.05,
    reg       = 0.01,
    seed      = SEED,
)

bpr.fit(
    train_users    = train_users,
    train_items    = train_items,
    user_pos_items = user_pos_items,
    n_items        = n_items,
    n_epochs       = 5,
    batch_size     = 4096,
    rng            = rng,
)

Epoch 1/5  log-likelihood: -0.2660
Epoch 2/5  log-likelihood: -0.1107
Epoch 3/5  log-likelihood: -0.0958
Epoch 4/5  log-likelihood: -0.0872
Epoch 5/5  log-likelihood: -0.0817


## 5 · Evaluate — Precision@K and nDCG@K

In [16]:
# Build test positives: user → set(positive movie_idxs in test)
test_df_pos = test_df[test_df['rating'] >= POS_THRESHOLD].copy()
test_df_pos['user_idx']  = test_df_pos['userId'].map(user_id_map)
test_df_pos['movie_idx'] = test_df_pos['movieId'].map(movie_id_map)
test_df_pos = test_df_pos.dropna(subset=['user_idx', 'movie_idx'])
test_df_pos[['user_idx', 'movie_idx']] = test_df_pos[['user_idx', 'movie_idx']].astype(int)

user_test_pos = (
    test_df_pos.groupby('user_idx')['movie_idx']
    .apply(set)
    .to_dict()
)

def evaluate_bpr(model, user_pos_train, user_pos_test,
                 n_items, k_values=(5, 10, 20), n_eval_users=2000):
    eval_users = list(user_pos_test.keys())
    if len(eval_users) > n_eval_users:
        eval_users = rng.choice(eval_users, size=n_eval_users, replace=False).tolist()

    metrics = {k: {'precision': [], 'ndcg': []} for k in k_values}

    for u in eval_users:
        pos_test  = user_pos_test.get(u, set())
        pos_train = user_pos_train.get(u, set())
        if not pos_test:
            continue

        scores    = model.score_batch(u, np.arange(n_items))  # (n_items,)
        # Exclude train positives
        for pidx in pos_train:
            scores[pidx] = -np.inf

        top_k_max = max(k_values)
        top_idxs  = np.argpartition(scores, -top_k_max)[-top_k_max:]
        top_idxs  = top_idxs[np.argsort(scores[top_idxs])[::-1]]

        for k in k_values:
            recs     = set(top_idxs[:k])
            hits     = recs & pos_test
            prec     = len(hits) / k

            # nDCG
            gains    = [1.0 if top_idxs[i] in pos_test else 0.0 for i in range(k)]
            dcg      = sum(g / np.log2(i + 2) for i, g in enumerate(gains))
            ideal    = sum(1.0 / np.log2(i + 2) for i in range(min(k, len(pos_test))))
            ndcg_val = dcg / ideal if ideal > 0 else 0.0

            metrics[k]['precision'].append(prec)
            metrics[k]['ndcg'].append(ndcg_val)

    print(f'Evaluated on {len(eval_users)} users\n')
    for k in k_values:
        p = np.mean(metrics[k]['precision'])
        n = np.mean(metrics[k]['ndcg'])
        print(f'k={k:2d}  Precision@k={p:.4f}   nDCG@k={n:.4f}')
    return metrics


metrics = evaluate_bpr(bpr, user_pos_items, user_test_pos, n_items)

Evaluated on 2000 users

k= 5  Precision@k=0.0429   nDCG@k=0.0421
k=10  Precision@k=0.0420   nDCG@k=0.0423
k=20  Precision@k=0.0400   nDCG@k=0.0417


## 6 · Sample Recommendations

In [17]:
movies_by_id = movies_df.set_index('movieId')

def get_bpr_recs(user_id, n=10):
    """Top-n BPR recommendations for a MovieLens user."""
    if user_id not in user_id_map:
        print(f'User {user_id} not found.')
        return pd.DataFrame()

    u_idx  = user_id_map[user_id]
    scores = bpr.score_batch(u_idx, np.arange(n_items))

    # Exclude already-seen items (train positives)
    for pidx in user_pos_items.get(u_idx, set()):
        scores[pidx] = -np.inf

    top_n = np.argsort(scores)[::-1][:n]
    rows  = []
    for midx in top_n:
        mid = idx_to_movie.get(int(midx))
        if mid and mid in movies_by_id.index:
            r = movies_by_id.loc[mid]
            rows.append({
                'title':      r['title'],
                'genres':     r['genres'],
                'year':       r.get('year'),
                'bpr_score':  round(float(scores[midx]), 4),
                'avg_rating': r.get('avg_rating'),
            })
    return pd.DataFrame(rows)


print('Top 10 BPR recommendations for user 1:')
get_bpr_recs(user_id=1, n=10)

Top 10 BPR recommendations for user 1:


,title,genres,year,bpr_score,avg_rating
0,"Graduate, The (1967)",Comedy|Drama|Romance,1967.0,3.2384,4.031101
1,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0,3.1914,4.283678
2,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,2001.0,3.1824,4.089757
3,American History X (1998),Crime|Drama,1998.0,3.1671,4.138910
4,"Lock, Stock & Two Smoking Barrels (1998)",Comedy|Crime|Thriller,1998.0,3.1456,4.026035
5,Taxi Driver (1976),Crime|Drama|Thriller,1976.0,3.1425,4.082552
6,Schindler's List (1993),Drama|War,1993.0,3.1408,4.246764
7,Die Hard (1988),Action|Crime|Thriller,1988.0,3.1005,3.932095
8,Blade Runner (1982),Action|Sci-Fi|Thriller,1982.0,3.0996,4.114852
9,Traffic (2000),Crime|Drama|Thriller,2000.0,3.0725,3.797256


## 7 · Hybrid: BPR + Content-Based

Same adaptive alpha strategy as notebook 06, but with BPR scores instead of SGD-MF.

In [18]:
from sklearn.preprocessing import normalize as sk_normalize
import scipy.sparse as sp

with open(MODELS / 'cb_model.pkl', 'rb') as f:
    cb_model = pickle.load(f)

# Actual key names in cb_model.pkl (verified):
#   movie_feature_matrix  – sparse (n_movies, n_feats)  ← NOT pre-normalised; normalise here
#   movie_idx_lookup      – {movie_id → row in feature matrix}  ← keyed by movie_id!
#   idx_to_movie_id       – {row → movie_id}
feat_mat   = sk_normalize(cb_model['movie_feature_matrix'], norm='l2')
mid_to_row = cb_model['movie_idx_lookup']    # {movie_id → row}
midpoint   = cb_model.get('rating_midpoint', 3.0)


def build_cb_profile(rated_dict):
    """Weighted average of feature vectors. rated_dict = {movie_id: rating}."""
    positions, weights = [], []
    for mid, rating in rated_dict.items():
        row = mid_to_row.get(mid)
        if row is None:
            continue
        positions.append(row)
        weights.append(float(rating) - midpoint)
    if not positions:
        return None
    w = np.array(weights, dtype=np.float32)
    # w @ sparse_slice returns a numpy ndarray (not sparse) in scipy
    profile = np.asarray(w @ feat_mat[positions]).ravel()
    return profile / (np.abs(w).sum() + 1e-9)


def get_bpr_hybrid_recs(user_id, n=10):
    """alpha * BPR + (1-alpha) * CB cosine similarity."""
    if user_id not in user_id_map:
        return pd.DataFrame()

    u_idx      = user_id_map[user_id]
    user_hist  = ratings_df[ratings_df['userId'] == user_id]
    n_rated    = len(user_hist)
    alpha      = 0.3 if n_rated < 50 else (0.6 if n_rated < 200 else 0.8)
    rated_dict = dict(zip(user_hist['movieId'], user_hist['rating']))
    profile    = build_cb_profile(rated_dict)

    bpr_scores = bpr.score_batch(u_idx, np.arange(n_items))

    if profile is not None:
        pv       = sp.csr_matrix(profile.reshape(1, -1))
        all_sims = (feat_mat @ pv.T).toarray().ravel()   # sparse @ sparse → sparse → dense

        # Build cb_scores array in movie_idx space
        cb_scores = np.full(n_items, 3.5, dtype=np.float32)
        for midx in range(n_items):
            mid = idx_to_movie.get(midx)
            if mid is not None:
                row = mid_to_row.get(mid)
                if row is not None and row < len(all_sims):
                    cb_scores[midx] = 0.2333 * float(all_sims[row]) + 3.0967
    else:
        cb_scores = bpr_scores.copy()
        alpha = 1.0

    hybrid = alpha * bpr_scores + (1 - alpha) * cb_scores

    # Exclude train positives
    for pidx in user_pos_items.get(u_idx, set()):
        hybrid[pidx] = -np.inf

    top_n = np.argsort(hybrid)[::-1][:n]
    rows  = []
    for midx in top_n:
        mid = idx_to_movie.get(int(midx))
        if mid and mid in movies_by_id.index:
            r = movies_by_id.loc[mid]
            rows.append({
                'title':        r['title'],
                'genres':       r['genres'],
                'year':         r.get('year'),
                'hybrid_score': round(float(hybrid[midx]), 4),
                'bpr_score':    round(float(bpr_scores[midx]), 4),
                'cb_score':     round(float(cb_scores[midx]), 4),
                'avg_rating':   r.get('avg_rating'),
            })
    return pd.DataFrame(rows)


print('Top 10 BPR-Hybrid for user 1:')
get_bpr_hybrid_recs(user_id=1, n=10)

Top 10 BPR-Hybrid for user 1:


,title,genres,year,hybrid_score,bpr_score,cb_score,avg_rating
0,"Graduate, The (1967)",Comedy|Drama|Romance,1967.0,3.2206,3.2384,3.1939,4.031101
1,American History X (1998),Crime|Drama,1998.0,3.1699,3.1671,3.1742,4.138910
2,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0,3.1612,3.1914,3.1160,4.283678
3,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,2001.0,3.1552,3.1824,3.1143,4.089757
4,Schindler's List (1993),Drama|War,1993.0,3.1528,3.1408,3.1707,4.246764
5,Taxi Driver (1976),Crime|Drama|Thriller,1976.0,3.1522,3.1425,3.1668,4.082552
6,"Lock, Stock & Two Smoking Barrels (1998)",Comedy|Crime|Thriller,1998.0,3.1409,3.1456,3.1339,4.026035
7,Traffic (2000),Crime|Drama|Thriller,2000.0,3.1103,3.0725,3.1669,3.797256
8,Die Hard (1988),Action|Crime|Thriller,1988.0,3.1074,3.1005,3.1178,3.932095
9,Blade Runner (1982),Action|Sci-Fi|Thriller,1982.0,3.1053,3.0996,3.1140,4.114852


## 8 · Save Model

In [19]:
import yaml

with open(MODELS / 'bpr_model.pkl', 'wb') as f:
    pickle.dump(bpr, f)

k10_p = float(np.mean(metrics[10]['precision']))
k10_n = float(np.mean(metrics[10]['ndcg']))

summary = {
    'model':       'BPR-MF',
    'n_factors':   bpr.U.shape[1],
    'lr':          bpr.lr,
    'reg':         bpr.reg,
    'precision@10': round(k10_p, 4),
    'ndcg@10':      round(k10_n, 4),
}
with open(MODELS / 'bpr_results.yaml', 'w') as f:
    yaml.dump(summary, f, default_flow_style=False)

print('Saved: src/models/bpr_model.pkl')
print('       src/models/bpr_results.yaml')
print()
print(yaml.dump(summary))

Saved: src/models/bpr_model.pkl
       src/models/bpr_results.yaml

lr: 0.05
model: BPR-MF
n_factors: 64
ndcg@10: 0.0423
precision@10: 0.042
reg: 0.01

